# week5_hw 東元電機新聞文字探勘

In [17]:
import jieba
import pandas as pd
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import timedelta, date
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def is_chinese(uchar):         
    if u'\u4e00' <= uchar<=u'\u9fff':
        return True
    else:
        return False

In [3]:
#專有名詞
with open('ProperN.txt', 'rt',encoding="utf-8") as txt:
    ProperN= txt.read()
ProperN = ProperN.split("\n")
for w in ProperN:
    jieba.add_word(w)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/bf/kxrhqhkd3s1263kmtf84x8zw0000gn/T/jieba.cache
Loading model cost 0.788 seconds.
Prefix dict has been built succesfully.


In [4]:
#停用詞
with open('StopWords.txt', 'rt',encoding="utf-8") as txt:
    stopword = txt.read()
stopword = stopword.split("\n")

In [5]:
def textMining(file):
    font = r"/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf/SimHei.ttf"
    #open file
    with open("./News/" + file + ".txt", 'rt',encoding="utf-8") as txt:
        data = txt.read()
    #text segmentation
    seg_list = jieba.cut(data, cut_all=False)
    seg_list = list(seg_list)
    filter_list = []
    wordDict = {}
    jump = False
    for word in seg_list:
        for s in word:
            if not is_chinese(s):
                jump = True
                break
        if not jump:
            filter_list.append(word)
        jump = False
    return filter_list

In [6]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2017, 1, 6)
end_date = date(2018, 10, 1)
dates = []
for single_date in daterange(start_date, end_date):
    dates.append(single_date.strftime("%Y%m%d"))

In [7]:
def get_join_list(file_date):
    try:
        filter_list = textMining(file_date)
        join_list = " ".join(filter_list)
        return join_list
    except:
        print("no news on " + file_date)

In [8]:
corpus = []
news_dates = []
for date in dates:
    news = get_join_list(date)
    if news != None:
        print("append news of date " + date)
        news_dates.append(date)
        corpus.append(news)

append news of date 20170106
no news on 20170107
no news on 20170108
append news of date 20170109
no news on 20170110
no news on 20170111
no news on 20170112
no news on 20170113
no news on 20170114
no news on 20170115
no news on 20170116
no news on 20170117
no news on 20170118
no news on 20170119
no news on 20170120
no news on 20170121
no news on 20170122
no news on 20170123
no news on 20170124
append news of date 20170125
append news of date 20170126
append news of date 20170127
no news on 20170128
no news on 20170129
no news on 20170130
no news on 20170131
no news on 20170201
no news on 20170202
no news on 20170203
no news on 20170204
no news on 20170205
no news on 20170206
no news on 20170207
no news on 20170208
no news on 20170209
no news on 20170210
no news on 20170211
no news on 20170212
no news on 20170213
no news on 20170214
no news on 20170215
no news on 20170216
append news of date 20170217
no news on 20170218
no news on 20170219
no news on 20170220
append news of date 201702

append news of date 20180302
no news on 20180303
no news on 20180304
no news on 20180305
no news on 20180306
append news of date 20180307
no news on 20180308
no news on 20180309
no news on 20180310
no news on 20180311
append news of date 20180312
no news on 20180313
no news on 20180314
no news on 20180315
append news of date 20180316
no news on 20180317
no news on 20180318
no news on 20180319
no news on 20180320
no news on 20180321
no news on 20180322
no news on 20180323
no news on 20180324
no news on 20180325
no news on 20180326
append news of date 20180327
append news of date 20180328
append news of date 20180329
no news on 20180330
no news on 20180331
no news on 20180401
no news on 20180402
no news on 20180403
no news on 20180404
no news on 20180405
no news on 20180406
no news on 20180407
no news on 20180408
no news on 20180409
append news of date 20180410
no news on 20180411
no news on 20180412
no news on 20180413
no news on 20180414
no news on 20180415
no news on 20180416
no news 

In [9]:
len(corpus)

99

In [10]:
# tfidf
vectorizer = TfidfVectorizer(max_df = 0.9, min_df = 0.2)
tfidf = vectorizer.fit_transform(corpus)

In [11]:
words = vectorizer.get_feature_names()
print("len(words): ", len(words))

len(words):  113


In [12]:
weight = tfidf.T.toarray()
len(weight)

113

In [13]:
print("tfidf.shape: ", tfidf.shape)

tfidf.shape:  (99, 113)


In [39]:
# check each document
for i in range(len(corpus)):
    print('----{0} NEWS----'.format(news_dates[i]))
    for j in range(len(words)):
        if tfidf[i,j] > 0.2:
              print(words[j], tfidf[i,j], i ,j)

----20170106 NEWS----
企業 0.22093658187661444 0 6
增加 0.22799144588375525 0 27
東元電機 0.4524516161865815 0 67
邱純枝 0.28580320444143864 0 101
馬達 0.207959067580104 0 112
----20170109 NEWS----
技術 0.20411620207309703 1 45
推出 0.5065301705736375 1 51
智慧 0.21519327803345187 1 62
目前 0.21519327803345187 1 76
研發 0.2181498122438444 1 80
空調 0.26190223359406883 1 83
管理系 0.26190223359406883 1 84
能源 0.23428529541884055 1 90
銷售 0.25749726490776137 1 103
----20170125 NEWS----
今年 0.40096775203476853 2 3
以上 0.2901973533417942 2 4
優勢 0.4389588706971517 2 9
去年 0.3511518061388616 2 15
同時 0.22332801892372414 2 22
發展 0.27050456102712317 2 75
集團 0.28571985886511103 2 107
----20170126 NEWS----
今年 0.5017794155667806 3 3
以上 0.2905272209446819 3 4
億元 0.29083318037765127 3 8
公司 0.22459706474667487 3 11
其中 0.2314952156811919 3 12
去年 0.4614106359518751 3 15
開始 0.24303742625092617 3 104
----20170127 NEWS----
台灣 0.24896449757487138 4 19
推動 0.26082446166180834 4 52
智慧 0.7312755897311185 4 62
產業 0.2697462335722881 4 74
發展 0.2

投入 0.3153943357414968 65 46
產品 0.23195972433171477 65 73
產業 0.2500107874543142 65 74
發展 0.39536773918185836 65 75
開發 0.2787000039055539 65 105
----20180307 NEWS----
提升 0.21151995310196472 66 54
邱純枝 0.23340732405509113 66 101
馬達 0.7642540715131954 66 112
----20180312 NEWS----
技術 0.2512347858354906 67 45
智能 0.3223602580224324 67 63
東元電機 0.2028943044929988 67 67
產品 0.23309534863805403 67 73
研發 0.44751323676163957 67 80
集團 0.20982541548735537 67 107
馬達 0.48959263613716253 67 112
----20180316 NEWS----
全球 0.3530625229564963 68 10
台灣 0.2667868014965428 68 19
成為 0.40989450812914735 68 43
技術 0.3303499578522289 68 45
推出 0.40989450812914735 68 51
產品 0.30649831527929267 68 73
發展 0.3482775478204114 68 75
製造 0.3791768447626431 68 94
----20180327 NEWS----
整合 0.42417455648861385 69 58
智慧 0.7089746238809009 69 62
系統 0.27663052176731123 69 86
----20180328 NEWS----
國際 0.2751230630335381 70 25
市場 0.2809374515717613 70 34
東元電機 0.2935422986366079 70 67
生產 0.23967222603331756 70 72
馬達 0.4249980742427934 70 1

In [15]:
df = pd.DataFrame(weight, index = words, columns = news_dates)
df

,20170106,20170109,20170125,20170126,20170127,20170217,20170221,20170321,20170328,20170415,...,20180705,20180709,20180712,20180718,20180801,20180813,20180822,20180911,20180927,20180928
中心,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.135412,0.000000,0.000000,...,0.217215,0.000000,0.000000,0.000000,0.000000,0.000000,0.161034,0.000000,0.000000,0.000000
主要,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.064904,0.000000,...,0.059492,0.090151,0.000000,0.000000,0.081837,0.000000,0.000000,0.043092,0.000000,0.000000
事業,0.168535,0.000000,0.000000,0.086932,0.000000,0.000000,0.000000,0.142862,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
今年,0.194561,0.159490,0.400968,0.501779,0.000000,0.096738,0.141385,0.082462,0.048103,0.000000,...,0.000000,0.000000,0.000000,0.466265,0.030327,0.000000,0.000000,0.095813,0.000000,0.049212
以上,0.000000,0.000000,0.290197,0.290527,0.000000,0.140027,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.096714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
以及,0.000000,0.120741,0.000000,0.000000,0.091179,0.439411,0.000000,0.000000,0.000000,0.000000,...,0.066760,0.000000,0.000000,0.000000,0.045917,0.000000,0.000000,0.048356,0.134492,0.074512
企業,0.220937,0.000000,0.000000,0.075974,0.000000,0.439411,0.053517,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.141082,0.000000,0.068876,0.000000,0.000000,0.000000,0.000000,0.000000
使用,0.000000,0.133247,0.000000,0.000000,0.000000,0.000000,0.059061,0.137787,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
億元,0.169152,0.000000,0.000000,0.290833,0.069808,0.112140,0.122921,0.000000,0.055762,0.000000,...,0.000000,0.000000,0.000000,0.270249,0.000000,0.853618,0.113677,0.481291,0.000000,0.000000
優勢,0.000000,0.000000,0.438959,0.000000,0.049445,0.000000,0.000000,0.135412,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [41]:
similarity = cosine_similarity(tfidf, tfidf)
type(similarity)

numpy.ndarray

In [50]:
similar_news_index = []
news_index = 0
for index1 in range(len(similarity)):
    news_index += 1
    for index2 in range(news_index, len(similarity[index1])):
        if 0.8 < sim[index1][index2] < 0.99:
            similar_news_index.append((index1, index2))

In [51]:
for i in similar_news_index:
    print(news_dates[i[0]], news_dates[i[1]])

20170605 20170731
20170720 20170724
20170814 20180514
20170814 20180813
20170930 20171010
20171104 20171106
20171114 20171115
20171207 20171219
20180514 20180813
20180705 20180709
20180927 20180928


In [53]:
# check each document
for index in similar_news_index:
    print('----{0} NEWS WORDS----'.format(news_dates[index[0]]))
    i = similar_news_index[0]
    for j in range(len(words)):
        if tfidf[i,j] > 0.2:
              print(words[j], tfidf[i,j], i ,j)

----20170605 NEWS WORDS----


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all().